In [1]:
# file: reuse_finetuned_model.py

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def load_finetuned_gpt2(model_dir):
    """
    Loads the fine-tuned GPT-2 model and tokenizer from a directory 
    and places the model in eval mode.
    """
    # 1) Load the tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_dir)
    
    # 2) Load the model
    model = GPT2LMHeadModel.from_pretrained(model_dir)
    
    # 3) Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    return model, tokenizer

def generate_remainder(model, tokenizer, partial_text, max_new_tokens=50):
    """
    Generates the conversation remainder (or continuation) given a partial snippet 
    using the fine-tuned GPT-2 model.
    """
    device = next(model.parameters()).device
    # Convert partial text to token IDs
    inputs = tokenizer(partial_text, return_tensors="pt").to(device)

    with torch.no_grad():
        # Generate tokens beyond the partial snippet
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,  # how many additional tokens to produce
            do_sample=False,                # or True for sampling
            pad_token_id=tokenizer.eos_token_id  # GPT-2 doesn't have real pad, so eos is used
        )
    # Decode the entire sequence
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return generated_text


c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\anaconda3\envs\ai-backend\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:

    # Example usage
model_dir = "gpt2_conversation_predict_model"  # the folder where you saved your fine-tuned model
model, tokenizer = load_finetuned_gpt2(model_dir)
    

    


In [7]:
test_partial = "Hello, I'm Ashan. I am calling on behalf of Bank of Ceylon Sri Lanka."
remainder_prediction = generate_remainder(model, tokenizer, test_partial)
    

In [8]:
print("=== PARTIAL ===")
print(test_partial)
print("\n=== MODEL'S COMPLETION ===")
print(remainder_prediction)

=== PARTIAL ===
Hello, I'm Ashan. I am calling on behalf of Bank of Ceylon Sri Lanka.

=== MODEL'S COMPLETION ===
Hello, I'm Ashan. I am calling on behalf of Bank of Ceylon Sri Lanka.
I'm calling from the Bank of Ceylon to ask you to provide us with a detailed account number and a detailed account number for your account.
Can you provide more details about the account you're using and the account number you're using?


### Test Model

In [9]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import pickle  # for saving tokenizer or other objects

In [10]:
def build_convo_pairs(csv_path, partial_ratio=0.5):
    df = pd.read_csv(csv_path)
    data_pairs = []
    for convo_id, group in df.groupby("CONVERSATION_ID"):
        group_sorted = group.sort_values("CONVERSATION_STEP")
        texts = group_sorted["TEXT"].tolist()

        full_str = "\n".join(texts).strip()

        if len(texts) < 2:
            continue  # Skip if only one message in conversation

        cutoff = max(1, min(len(texts) - 1, int(len(texts) * partial_ratio)))  # Ensures remainder isn't empty
        partial_list = texts[:cutoff]
        remainder_list = texts[cutoff:]

        partial_str = "\n".join(partial_list).strip()
        remainder_str = "\n".join(remainder_list).strip() if remainder_list else ""

        data_pairs.append({
            "full": full_str,
            "partial": partial_str,
            "remainder": remainder_str
        })

    return data_pairs

In [11]:
csv_path = "C:/Users/DELL/Desktop/VOIP_Phishing_Attacks/Repos/convoPredict/conversation-prediction/FINAL_DATASET2.csv"  # <-- update with your CSV path
data_pairs = build_convo_pairs(csv_path, partial_ratio=0.5)

In [12]:
data_pairs

[{'full': "Hello, this is [Your Name]'s personal assistant. How may I assist you today?\nHi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.\nHi Sam, it's great to hear of your interest in the photography workshop. I'd be happy to help with any questions you have.\nThanks! I was wondering about the skill level required for participants. I'm fairly new to photography.\nThe workshop is designed to accommodate all skill levels, from beginners to more experienced photographers. [Org Name] aims to ensure everyone can learn and grow, regardless of their starting point.\nThat sounds perfect. What's the registration process?\nYou can register through our website. I can guide you through the steps if you'd like, or send you a direct link to the registration page.\nA direct link would be great. Can you also tell me about the workshop fee?\nCertainly, the fee for the workshop is $200, which includes all mate

In [13]:
full_convo = [pair["full"] for pair in data_pairs]
partial_values = [pair["partial"] for pair in data_pairs]
remainder_values = [pair["remainder"] for pair in data_pairs]

In [15]:
partial_values[0]

"Hello, this is [Your Name]'s personal assistant. How may I assist you today?\nHi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.\nHi Sam, it's great to hear of your interest in the photography workshop. I'd be happy to help with any questions you have.\nThanks! I was wondering about the skill level required for participants. I'm fairly new to photography.\nThe workshop is designed to accommodate all skill levels, from beginners to more experienced photographers. [Org Name] aims to ensure everyone can learn and grow, regardless of their starting point.\nThat sounds perfect. What's the registration process?"

In [17]:
for convo in partial_values:
    partial_convo = convo
    remainder_prediction = generate_remainder(model, tokenizer, partial_convo)
    print("PARTIAL CONVO")
    print(convo)
    print("REMINDER CONVO")
    print(remainder_prediction)
    print("==========================")


PARTIAL CONVO
Hello, this is [Your Name]'s personal assistant. How may I assist you today?
Hi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.
Hi Sam, it's great to hear of your interest in the photography workshop. I'd be happy to help with any questions you have.
Thanks! I was wondering about the skill level required for participants. I'm fairly new to photography.
The workshop is designed to accommodate all skill levels, from beginners to more experienced photographers. [Org Name] aims to ensure everyone can learn and grow, regardless of their starting point.
That sounds perfect. What's the registration process?
REMINDER CONVO
Hello, this is [Your Name]'s personal assistant. How may I assist you today?
Hi, I'm Sam. I saw an ad about a photography workshop hosted by [Org Name] next month. I'm interested in registering but had a few questions.
Hi Sam, it's great to hear of your interest in the ph